# Overview
This script allows you to scrape a website to download documents. Please use this with caution, be sure to have permission to do this before you do. 

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import ssl
from urllib.parse import urljoin
import time

# Set the url for the website here

website_url = 'https://example.com'

# Set the output directory for downloaded files
output_directory = 'files'
download_timeout = 6  # Timeout in seconds
max_file_size = 120 * 1024 * 1024  # Maximum file size to download in bytes (120MB in this case)

# Create a output directory if one is already created.
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Important
You may need to create custom SSL certificates but I'm not sure how to do that yet but will figure it out and update this notebook

In [ ]:
# Create a custom SSL context
context = ssl.create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE

# Function to download a file
def download_file(file_url, file_name, max_retries=3):
    for attempt in range(max_retries):
        try:
            with requests.get(file_url, stream=True, verify=False, timeout=download_timeout) as r:
                r.raise_for_status()
                if int(r.headers.get('content-length', 0)) > max_file_size:
                    print(f"File {file_name} is too large. Skipping...")
                    return False
                with open(file_name, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f'Successfully downloaded: {file_name}')
            return True
        except (requests.exceptions.RequestException, requests.exceptions.Timeout) as e:
            print(f"Attempt {attempt + 1} failed for {file_url}: {e}")
            if attempt < max_retries - 1:
                print(f"Retrying in 2 seconds...")
                time.sleep(2)
            else:
                print(f"Max retries reached. Moving to next file.")
    return False

response = requests.get(website_url, verify=False)
soup = BeautifulSoup(response.content, 'html.parser')

for link in soup.find_all('a', href=True):
    file_url = link['href']
    # Convert relative URL to absolute URL
    file_url = urljoin(website_url, file_url)
    if file_url.endswith(('.pdf', '.docx', '.xlsx')):  # Add other document types as needed
        file_name = os.path.join(output_directory, os.path.basename(file_url))
        download_file(file_url, file_name)

print("Download process completed.")

# Final Part
Now, to download the file. I used Shutil to make a zip file. 

In [ ]:
# create a zip file from Publications

import shutil
shutil.make_archive('/content/folder', 'zip', '/content/folder')
print("Zip file created!")

Zip file created!
